In [1]:
import numpy as np
from keras.layers import Reshape, Lambda, Flatten, Activation, Conv2D, Conv2DTranspose, Dense, Input, Subtract, Add, Multiply
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Sequential, Model
from keras.engine.network import Network
from keras.optimizers import Adadelta
import keras.backend as K
import tensorflow as tf

Using TensorFlow backend.


In [2]:
global_shape = (256,256,3)
local_shape = (128,128,3)
optimizer = Adadelta()

# Create the GANs
From scratch, and combining each neural net together, until we create a master brain

### Primative Generator Net
This does not include the masks, we only define the images being inputted. We will add the masks later (turning into an augmented net)

In [3]:
def model_generator(input_shape=(256, 256, 3)):
    in_layer = Input(shape=input_shape)

    model = Conv2D(64, kernel_size=5, strides=1, padding='same',
                     dilation_rate=(1, 1))(in_layer)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)

    model = Conv2D(128, kernel_size=3, strides=2,
                     padding='same', dilation_rate=(1, 1))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Conv2D(128, kernel_size=3, strides=1,
                     padding='same', dilation_rate=(1, 1))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)

    model = Conv2D(256, kernel_size=3, strides=2,
                     padding='same', dilation_rate=(1, 1))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Conv2D(256, kernel_size=3, strides=1,
                     padding='same', dilation_rate=(1, 1))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Conv2D(256, kernel_size=3, strides=1,
                     padding='same', dilation_rate=(1, 1))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)

    model = Conv2D(256, kernel_size=3, strides=1,
                     padding='same', dilation_rate=(2, 2))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Conv2D(256, kernel_size=3, strides=1,
                     padding='same', dilation_rate=(4, 4))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Conv2D(256, kernel_size=3, strides=1,
                     padding='same', dilation_rate=(8, 8))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Conv2D(256, kernel_size=3, strides=1,
                     padding='same', dilation_rate=(16, 16))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)

    model = Conv2D(256, kernel_size=3, strides=1,
                     padding='same', dilation_rate=(1, 1))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Conv2D(256, kernel_size=3, strides=1,
                     padding='same', dilation_rate=(1, 1))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)

    model = Conv2DTranspose(128, kernel_size=4, strides=2,
                              padding='same')(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Conv2D(128, kernel_size=3, strides=1,
                     padding='same', dilation_rate=(1, 1))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)

    model = Conv2DTranspose(64, kernel_size=4, strides=2,
                              padding='same')(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Conv2D(32, kernel_size=3, strides=1,
                     padding='same', dilation_rate=(1, 1))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)

    model = Conv2D(3, kernel_size=3, strides=1,
                     padding='same', dilation_rate=(1, 1))(model)
    model = BatchNormalization()(model)
    model = Activation('sigmoid')(model)
    model_gen = Model(inputs=in_layer, outputs=model)
    model_gen.name = 'Gener8tor'
    return model_gen

### Primative Discriminator Net
This accounts for the masks and input images, but is not connected to anything else.

In [4]:
def model_discriminator(global_shape=(256, 256, 3), local_shape=(128, 128, 3)):
    def crop_image(img, crop):
        return tf.image.crop_to_bounding_box(img,
                                             crop[1],
                                             crop[0],
                                             crop[3] - crop[1],
                                             crop[2] - crop[0])

    in_pts = Input(shape=(4,), dtype='int32')
    cropping = Lambda(lambda x: K.map_fn(lambda y: crop_image(y[0], y[1]), elems=x, dtype=tf.float32),
                      output_shape=local_shape)
    g_img = Input(shape=global_shape)
    l_img = cropping([g_img, in_pts])

    # Local Discriminator
    x_l = Conv2D(64, kernel_size=5, strides=2, padding='same')(l_img)
    x_l = BatchNormalization()(x_l)
    x_l = Activation('relu')(x_l)
    x_l = Conv2D(128, kernel_size=5, strides=2, padding='same')(x_l)
    x_l = BatchNormalization()(x_l)
    x_l = Activation('relu')(x_l)
    x_l = Conv2D(256, kernel_size=5, strides=2, padding='same')(x_l)
    x_l = BatchNormalization()(x_l)
    x_l = Activation('relu')(x_l)
    x_l = Conv2D(512, kernel_size=5, strides=2, padding='same')(x_l)
    x_l = BatchNormalization()(x_l)
    x_l = Activation('relu')(x_l)
    x_l = Conv2D(512, kernel_size=5, strides=2, padding='same')(x_l)
    x_l = BatchNormalization()(x_l)
    x_l = Activation('relu')(x_l)
    x_l = Flatten()(x_l)
    x_l = Dense(1024, activation='relu')(x_l)

    # Global Discriminator
    x_g = Conv2D(64, kernel_size=5, strides=2, padding='same')(g_img)
    x_g = BatchNormalization()(x_g)
    x_g = Activation('relu')(x_g)
    x_g = Conv2D(128, kernel_size=5, strides=2, padding='same')(x_g)
    x_g = BatchNormalization()(x_g)
    x_g = Activation('relu')(x_g)
    x_g = Conv2D(256, kernel_size=5, strides=2, padding='same')(x_g)
    x_g = BatchNormalization()(x_g)
    x_g = Activation('relu')(x_g)
    x_g = Conv2D(512, kernel_size=5, strides=2, padding='same')(x_g)
    x_g = BatchNormalization()(x_g)
    x_g = Activation('relu')(x_g)
    x_g = Conv2D(512, kernel_size=5, strides=2, padding='same')(x_g)
    x_g = BatchNormalization()(x_g)
    x_g = Activation('relu')(x_g)
    x_g = Conv2D(512, kernel_size=5, strides=2, padding='same')(x_g)
    x_g = BatchNormalization()(x_g)
    x_g = Activation('relu')(x_g)
    x_g = Flatten()(x_g)
    x_g = Dense(1024, activation='relu')(x_g)

    x = Concatenate(axis=1)([x_l, x_g])
    x = Dense(1, activation='sigmoid')(x)
    model_disc = Model(inputs=[g_img, in_pts], outputs=x)
    model_disc.name = 'Discimi-hater'
    return model_disc

In [5]:
def view_models(model, filename):
    from keras.utils import plot_model
    plot_model(model, to_file=filename, show_shapes=True)

### Augmented Generator Net
We connect the masks now, turning the primitive net more advanced

In [6]:
def full_gen_layer(full_img, mask, ones):
    from keras.layers import Concatenate

    # grab the inverse mask, that only shows the masked areas
    # 1 - mask
    inverse_mask = Subtract()([ones, mask])

    # which outputs the erased_image as input
    # full_img * (1 - mask)
    erased_image = Multiply()([full_img, inverse_mask])

    # view our net
    gen_model = model_generator(global_shape)
    # print(gen_model)

    # pass in the erased_image as input
    gen_model = gen_model(erased_image)
    # print(gen_model)

    gen_brain = Model(inputs=[full_img, mask, ones], outputs=gen_model)
    # print(gen_brain)
    view_models(gen_brain, 'summaries/gen_brain.png')

    gen_brain.compile(
        loss='mse',
        optimizer=optimizer
    )
    # gen_brain.summary()
    return gen_brain, gen_model

### Connected Discriminator Net
We connect the primitive discriminator net to the output of the augmented generator net

In [7]:
def full_disc_layer(global_shape, local_shape, full_img, clip_coords):
    # the discriminator side
    disc_model = model_discriminator(global_shape, local_shape)

    disc_model = disc_model([full_img, clip_coords])
    disc_model
    # print(disc_model)

    disc_brain = Model(inputs=[full_img, clip_coords], outputs=disc_model)
    disc_brain.compile(loss='binary_crossentropy',
                        optimizer=optimizer)
    # disc_brain.summary()
    view_models(disc_brain, 'summaries/disc_brain.png')
    return disc_brain, disc_model

In [8]:

full_img = Input(shape=global_shape)
clip_img = Input(shape=local_shape)
mask = Input(shape=(global_shape[0], global_shape[1], 1))
ones = Input(shape=(global_shape[0], global_shape[1], 1))
clip_coords = Input(shape=(4,), dtype='int32')

gen_brain, gen_model = full_gen_layer(full_img, mask, ones)
disc_brain, disc_model = full_disc_layer(global_shape, local_shape, full_img, clip_coords)

print(gen_brain)
print(disc_brain)

print(gen_model)
print(disc_model)

Tensor("Gener8tor/activation_17/Sigmoid:0", shape=(?, ?, ?, 3), dtype=float32)
Tensor("Discimi-hater/dense_3/Sigmoid:0", shape=(?, 1), dtype=float32)


### Connect the Neural Nets

In [9]:
alpha = 0.0004

# the final brain
disc_model.trainable = False
connected_disc = Model(inputs=[full_img, clip_coords], outputs=disc_model)
connected_disc.name = 'Connected-Discrimi-Hater'
print(connected_disc)

brain = Model(inputs=[full_img, mask, ones, clip_coords], outputs=[gen_model, connected_disc([gen_model, clip_coords])])
brain.compile(loss=['mse', 'binary_crossentropy'],
                      loss_weights=[1.0, alpha], optimizer=optimizer)
brain.summary()
view_models(brain, 'summaries/brain.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
subtract_1 (Subtract)           (None, 256, 256, 1)  0           input_4[0][0]                    
                                                                 input_3[0][0]                    
__________

# Setup the Image Preprocessor
Using a memory-efficient Python generator

In [10]:
from tensorflow.python.lib.io import file_io
from google.cloud import storage
import google
import cv2
from PIL import Image

# creds, _ = google.auth.default()
client = storage.Client()
bucket = client.bucket('lsun-roomsets')

/Users/kangzehuang/anaconda3/envs/ComputerVision/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/kangzehuang/anaconda3/envs/ComputerVision/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about 

In [11]:
# playable version of DataGenerator()
class FuckAround():
    # list the bucket and directory within
    bucketname = 'gs://lsun-roomsets'
    directory = 'images/bedroom_val/'
    
    # loop through all the files and view first X images (count)
    count = 0
    max_count = 10
    # store the raw img urls here
    img_urls = []
    for blob in bucket.list_blobs(prefix=directory):
        if count >= max_count:
            break
        print(blob.name)
        count += 1
        img_urls.append(blob.name)
        
    # store the resized images here
    images = []
    points = []
    masks = []
    
    # CONSTANTS
    # mask size limits
    hole_min = 64
    hole_max = 128
    # batch limits
    batch_size = 5
    max_batches = 3
    batch_count = 0
    # image sizes
    image_size = (256,256)
    local_size = (128,128)
    
    for idx, img_url in enumerate(img_urls):
        # we use tf...file_io.FileIO to grab the file
        with file_io.FileIO(f'{bucketname}/{img_url}', 'rb') as f:
            # and use PIL to convert into an RGB image
            img = Image.open(f).convert('RGB')
            # then convert the RGB image to an array so that cv2 can read it
            img = np.asarray(img, dtype="uint8")
            # resize images
            img_resized = cv2.resize(img, image_size)[:,:,::-1]
            # take a look at the images
            # cv2.imshow(f'image_{idx}_resized', img_resized)
            # cv2.waitKey(0)
            # cv2.destroyWindow(f'image_{idx}_resized')
            # add the resized photo to self.images
            images.append(img_resized)
            print(f'{idx}. Processing {img_url}')
            
            # now lets create the random points where we will apply a mask (erase parts of image)
            # recall that image_size=(256,256) and local_size=(128,128)
            x1 = np.random.randint(0, image_size[0] - local_size[0] + 1)
            y1 = np.random.randint(0, image_size[1] - local_size[1] + 1)
            x2, y2 = np.array([x1, y1]) + np.array(local_size)
            points.append([x1,y1,x2,y2])
            
            # and we also randomly generate width and height of those masks
            w, h = np.random.randint(hole_min, hole_max, 2)
            p1 = x1 + np.random.randint(0, local_size[0] - w)
            q1 = y1 + np.random.randint(0, local_size[1] - h)
            p2 = p1 + w
            q2 = q1 + h
            # now create the array of zeros
            m = np.zeros((image_size[0], image_size[1], 1), dtype=np.uint8)
            # everywhere there should be the mask, make the value one (everywhere else is zero)
            m[q1:q2 + 1, p1:p2 + 1] = 1
            # finally append it to the self.masks
            masks.append(m)
            
            # print the batch of data when batch size reached
            if len(images) == batch_size:
                print(np.array(images).shape)
                print(np.array(points).shape)
                print(np.array(masks).shape)
                inputs = np.asarray(images, dtype=np.float32) / 255
                points = np.asarray(points, dtype=np.int32)
                masks = np.asarray(masks, dtype=np.float32)
                
                # reset
                images = []
                points = []
                masks = []
                batch_count += 1
                
            if batch_count > max_batches:
                break
            

images/bedroom_val/0/0/0/0/0/0/00000089629ce3ba87bae003073896ba01988dee.webp
images/bedroom_val/0/0/0/0/0/1/000001ec5684cb40f432f996f8a38e5d076114d8.webp
images/bedroom_val/0/0/0/0/0/3/0000036b25b1ae054cdf2e3ee954fe2d21db6ae0.webp
images/bedroom_val/0/0/0/0/0/b/00000b94a63ae2e1c08b4e7452d088156a9a8273.webp
images/bedroom_val/0/0/0/0/1/2/0000128037967f0d4b7ba748a80d5b248d1203f8.webp
images/bedroom_val/0/0/0/0/1/5/000015122516efa29c25870b6b60fafe2fae1513.webp
images/bedroom_val/0/0/0/0/1/c/00001c1755ac170e5382876232aec651f6bda841.webp
images/bedroom_val/0/0/0/0/2/3/000023924aa8e512db983cba65e30cc106123ce3.webp
images/bedroom_val/0/0/0/0/3/5/0000356acb787613fc8d8715cc6c182c05173535.webp
images/bedroom_val/0/0/0/0/3/f/00003f8ec7ff5d59865ab2b6fb58bc663ace3b23.webp
0. Processing images/bedroom_val/0/0/0/0/0/0/00000089629ce3ba87bae003073896ba01988dee.webp
1. Processing images/bedroom_val/0/0/0/0/0/1/000001ec5684cb40f432f996f8a38e5d076114d8.webp
2. Processing images/bedroom_val/0/0/0/0/0/3/000

### DataGenerator
Using a memory-efficient Python generator

In [21]:
class DataGenerator(object):
    # initialize by retreiving the photos
    def __init__(self, bucketname, input_dir, image_size, local_size):
        # bucketname = 'gs://lsun-roomsets'
        # input_dir = 'images/bedroom_train/'
        # image_size = (256,256)
        # local_size = (128,128)
        self.image_size = image_size
        self.local_size = local_size
        self.reset()
        self.img_file_list = []
        # for now we get max self.count photos and add them to self.img_file_list
        for blob in bucket.list_blobs(prefix=input_dir):
            self.img_file_list.append(blob.name)
            
    def __len__(self):
        return len(self.img_file_list)
    
    # we also track the preprocessed images, points, and masks
    def reset(self):
        self.images = []
        self.points = []
        self.masks = []
    
    # iterates over self.img_file_list and does preprocessing
    def flow(self, batch_size, hole_min=64, hole_max=128):
        np.random.shuffle(self.img_file_list)
        for idx, img_url in enumerate(self.img_file_list):
            # we use tf...file_io.FileIO to grab the file
            with file_io.FileIO(f'{bucketname}/{img_url}', 'rb') as f:
                # and use PIL to convert into an RGB image
                img = Image.open(f).convert('RGB')
                # then convert the RGB image to an array so that cv2 can read it
                img = np.asarray(img, dtype="uint8")
                # resize images
                img_resized = cv2.resize(img, self.image_size)[:,:,::-1]
                # take a look at the images
                # cv2.imshow(f'image_{idx}_resized', img_resized)
                # cv2.waitKey(0)
                # cv2.destroyWindow(f'image_{idx}_resized')
                # add the resized photo to self.images
                self.images.append(img_resized)

                # now lets create the random location (aka. X,Y points) where we will apply a mask (aka. erase parts of image)
                # recall that image_size=(256,256) and local_size=(128,128)
                x1 = np.random.randint(0, self.image_size[0] - self.local_size[0] + 1)
                y1 = np.random.randint(0, self.image_size[1] - self.local_size[1] + 1)
                x2, y2 = np.array([x1, y1]) + np.array(self.local_size)
                self.points.append([x1,y1,x2,y2])
                # and we also randomly generate width and height of those masks
                w, h = np.random.randint(hole_min, hole_max, 2)
                p1 = x1 + np.random.randint(0, self.local_size[0] - w)
                q1 = y1 + np.random.randint(0, self.local_size[1] - h)
                p2 = p1 + w
                q2 = q1 + h
                # now create the array of zeros
                m = np.zeros((self.image_size[0], self.image_size[1], 1), dtype=np.uint8)
                # everywhere there should be the mask, make the value one (everywhere else is zero)
                m[q1:q2 + 1, p1:p2 + 1] = 1
                # finally append it to the self.masks
                self.masks.append(m)

                # yeild the batch of data when batch size reached
                if len(self.images) == batch_size:
                    images = np.asarray(self.images, dtype=np.float32) / 255
                    points = np.asarray(self.points, dtype=np.int32)
                    masks = np.asarray(self.masks, dtype=np.float32)
                    self.reset()
                    yield images, points, masks

# Start the Training
With hyperparameters

In [ ]:
from keras.utils import generic_utils
import os

In [23]:
# hyperparameters
input_shape = (256, 256, 3)
local_shape = (128, 128, 3)
batch_size = 4
n_epochs = 2
tc = int(n_epochs * 0.18)
td = int(n_epochs * 0.02)
alpha = 0.0004
gen_img_count = 0

# input/output directories
bucketname = "gs://lsun-roomsets"
result_dir = "outputs/"
input_dir = "images/bedroom_val/"

# data generator
train_datagen = DataGenerator(bucketname, input_dir, input_shape[:2], local_shape[:2])

In [ ]:
# train over time
for epoch in range(n_epochs):
    # progress bar visualization (comment out in ML Engine)
    progbar = generic_utils.Progbar(len(train_datagen))
    for images, points, masks in train_datagen.flow(batch_size):
        # and the matrix of ones that we depend on in the neural net to inverse masks
        mask_inv = np.ones((len(images), input_shape[0], input_shape[1], 1))
        # generate the inputs (images)
        generated_img = gen_brain.predict([images, masks, mask_inv])
        # generate the labels
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        # the gen and disc losses
        g_loss = 0.0
        d_loss = 0.0
        # ______________________
        if epoch < tc:
            # set the gen loss
            g_loss = gen_brain.train_on_batch([images, points], valid)
        # ______________________
        else:
            # throw in real unedited images with label VALID
            d_loss_real = disc_brain.train_on_batch([images, points], valid)
            # throw in A.I. generated images with label FAKE
            d_loss_fake = disc_brain.train_on_batch([generated_img, points], fake)
            # combine and set the disc loss
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            # ______________________
            if epoch >= tc + td:
                # train the entire brain
                g_loss = brain.train_on_batch([images, masks, mask_inv, points], [images, valid])
                # and update the generator loss
                g_loss = g_loss[0] + alpha * g_loss[1]
        # progress bar visualization (comment out in ML Engine)
        progbar.add(images.shape[0], values=[("Disc Loss: ", d_loss), ("Gen mse: ", g_loss)])
        gen_img_count += 1
        # save the generated image
        last_img = generated_img[0]
        last_img[:,:,0] = last_img[:,:,0]*255
        last_img[:,:,1] = last_img[:,:,1]*255
        last_img[:,:,2] = last_img[:,:,2]*255
        dreamt_image = Image.fromarray(last_img.astype(int), 'RGB')
        dreamt_image.save(f"outputs/images/epoch_{gen_img_count}_image.png")
        
gen_model.save(os.path.join(result_dir, "generator.h5"))
disc_model.save(os.path.join(result_dir, "discriminator.h5"))

  4/300 [..............................] - ETA: 45:03 - Disc Loss: : 7.9712 - Gen mse: : 0.0318------
[[[[0.36589742 0.4796006  0.44764042]
   [0.36185306 0.41578954 0.3460058 ]
   [0.37443367 0.42118323 0.3720826 ]
   ...
   [0.36393288 0.4677691  0.37229434]
   [0.3426466  0.4257327  0.29105556]
   [0.44055435 0.43270466 0.46049324]]

  [[0.34002724 0.40354356 0.38993517]
   [0.3091823  0.30161405 0.2817053 ]
   [0.3567484  0.27552795 0.3383008 ]
   ...
   [0.35678348 0.32234716 0.32693538]
   [0.2518351  0.28943342 0.26035097]
   [0.4094386  0.39001936 0.45509684]]

  [[0.34217232 0.46607283 0.3471618 ]
   [0.31349322 0.32837093 0.27957046]
   [0.33640525 0.32222697 0.32603547]
   ...
   [0.29314953 0.37072426 0.35085097]
   [0.33512673 0.35648164 0.31695822]
   [0.41808552 0.39816713 0.44146225]]

  ...

  [[0.36135992 0.40068713 0.3354983 ]
   [0.25738204 0.25466618 0.30283841]
   [0.3530785  0.29712328 0.3140073 ]
   ...
   [0.29844412 0.27467063 0.25029653]
   [0.2867688  0.2567

 12/300 [>.............................] - ETA: 44:18 - Disc Loss: : 7.9712 - Gen mse: : 0.0376------
[[[[0.3503689  0.47131974 0.41943514]
   [0.343667   0.4061773  0.33580345]
   [0.33751556 0.38179937 0.3625638 ]
   ...
   [0.68485    0.6898206  0.5824831 ]
   [0.6107756  0.6116496  0.69307494]
   [0.6210017  0.5651629  0.5256727 ]]

  [[0.326141   0.38949484 0.36696884]
   [0.2751603  0.27070194 0.26896358]
   [0.30033877 0.25649887 0.31987974]
   ...
   [0.72365814 0.6607537  0.60292834]
   [0.5144404  0.7649419  0.565985  ]
   [0.58395463 0.53872764 0.5372266 ]]

  [[0.33208907 0.446341   0.3055503 ]
   [0.27823603 0.29609746 0.28720716]
   [0.3123147  0.31246525 0.288781  ]
   ...
   [0.7669569  0.85223776 0.69396263]
   [0.49137893 0.6304601  0.7637132 ]
   [0.5840204  0.6673381  0.4715996 ]]

  ...

  [[0.3746186  0.3840214  0.3396663 ]
   [0.24607956 0.2623305  0.25759062]
   [0.3166487  0.27878836 0.32891813]
   ...
   [0.30991638 0.3014236  0.2479616 ]
   [0.2746045  0.2377

 20/300 [=>............................] - ETA: 42:34 - Disc Loss: : 7.9712 - Gen mse: : 0.0361------
[[[[0.3563316  0.45519364 0.4088742 ]
   [0.34963843 0.40177232 0.33683735]
   [0.34500223 0.3767555  0.3653191 ]
   ...
   [0.32795337 0.41059804 0.32107684]
   [0.3300417  0.37702805 0.28387484]
   [0.43985987 0.39411676 0.43800193]]

  [[0.3246005  0.38955003 0.38406205]
   [0.2873537  0.28327107 0.2830343 ]
   [0.30425683 0.2635256  0.30795786]
   ...
   [0.32553145 0.25778633 0.29720494]
   [0.27167368 0.24783893 0.25150907]
   [0.3897106  0.34910053 0.415422  ]]

  [[0.3405178  0.43803388 0.31173682]
   [0.28677326 0.31971803 0.29878166]
   [0.3265509  0.2974176  0.30722687]
   ...
   [0.26082313 0.29660323 0.29376858]
   [0.264296   0.26848808 0.22831477]
   [0.39211643 0.3325753  0.41077384]]

  ...

  [[0.36568233 0.37203336 0.32879296]
   [0.25039956 0.27398324 0.28343955]
   [0.2830021  0.2858105  0.3394029 ]
   ...
   [0.2943837  0.33171362 0.27733997]
   [0.31851262 0.3104

# To Do List
1. Find out when to use model.train_on_batch() vs model.fit() vs model.fit_to_generator()
2. Find out how to save the last generated image of each epoch to Google Cloud Storage
3. Find out how to retrieve and display the accuracy & percision metrics during training
4. Re-write the iPython Notebook into a Python Module
5. Setup the arguements injection for ML Engine
6. Setup multi-GPU training on ML Engine
7. Test on ML Engine
8. Train fully on ML Engine

9. Maybe some data augmentation?